# Pipeline for selection of embeddings

**TABLE OF CONTENTS**
1. Imports
2. A list of sample questions, to which ground truth exists
3. Trying out three multilingual embedding to compare their performance with the ground truth
    * a) jinaai/jina-embeddings-v2-base-de
    * b) intfloat/multilingual-e5-large
    * c) T-Systems-onsite/cross-en-de-roberta-sentence-transformer

## 1. Imports

In [1]:
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"   
# The above env var is for c) T-Systems-onsite/cross-en-de-roberta-sentence-transformer. 
# Might not be needed on a different machine.
import pickle
from transformers import AutoModel  # needed for the following workaround: https://github.com/langchain-ai/langchain/issues/6080

/Users/Onema/anaconda3/envs/4llama_py11/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. A list of sample questions, to which ground truth exists

In [2]:
question_list = ["How do hamsters compare with marmots?",
                "Which animals are the most beautiful?",
                "Which body parts of birds make them capable of flight?",
                "Where do hamsters live?",
                "Is there a connection between climate and animal coloration?"]

## 3. Trying out three multilingual embeddings to compare their performance with the ground truth

In [3]:
# loading the document chunks of size 700

# with register
with open("../data/pickles/ed1_docs_with_register_700_200.pickle", 'rb') as f:
    docs_ed1_700_with_register = pickle.load(f)
    
# without register
with open("../data/pickles/ed1_docs_without_register_700_200.pickle", 'rb') as f:
    docs_ed1_700_without_register = pickle.load(f)

In [4]:
def test_embedding(file_name, vectordb, model_name, question_list = question_list, retrieve_no = 5, 
                   save_path = "../data/results/embeddings_evaluation", register = True):
    
    if register:
        regis_print = "with"
    else:
        regis_print = "without"
        
    path_to_save = f"{save_path}/{file_name}_{regis_print}_register.txt"
    
    with open(path_to_save, "w") as f:

        print(f"This is a trial with {model_name} and {regis_print} register")
        f.write(f"This is a trial with {model_name} and {regis_print} register\n")
        
        for question in question_list:

            print(f"\n\nQuestion: {question}")
            f.write(f"\n\nQuestion: {question}\n")

            identified_docs = vectordb.similarity_search(question, retrieve_no)

            for doc in identified_docs:
                print(f"\n{doc.page_content}\n(Metadata: {doc.metadata})")
                f.write(f"\n{doc.page_content}\n(Metadata: {doc.metadata})\n")

### a) jinaai/jina-embeddings-v2-base-de

In [5]:
from huggingface_hub import login
login(token = hf_logging_token)   # UPDATE THE TOKEN FOR RUNNING THE NOTEBOOKS

/Users/Onema/anaconda3/envs/4llama_py11/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/Onema/.cache/huggingface/token
Login successful


In [7]:
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-de', trust_remote_code=True) 

config.json: 100%|█████████████████████████| 1.18k/1.18k [00:00<00:00, 1.13MB/s]
configuration_bert.py: 100%|███████████████| 8.24k/8.24k [00:00<00:00, 11.5MB/s]
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
modeling_bert.py: 100%|████████████████████| 97.5k/97.5k [00:00<00:00, 1.12MB/s]
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
pytorch_model.bin: 100%|█████████████████████| 322M/322M [00:08<00:00, 36.2MB/s]
/Users/Onema/anaconda3/envs/4llama_py11/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: Type

In [9]:
model_name = "jinaai/jina-embeddings-v2-base-de"
model_kwargs = {'device': 'cpu'}
hf_jina = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs)

#### With register

In [10]:
vectordb_jina = FAISS.from_documents(documents = docs_ed1_700_with_register, 
                                embedding = hf_jina)

In [11]:
test_embedding(file_name = "jina", vectordb = vectordb_jina, 
               model_name = model_name, register = True)

This is a trial with jinaai/jina-embeddings-v2-base-de and with register


Question: How do hamsters compare with marmots?

8. Taf. I. II Die Murmelthiere und die Hamster baden in Rücksicht ihres Körperbaues und ihrer Lebensart vieles mit einander gemein. Nur bewohnt der Hamster mehr nordliche Gegenden und flaches Land; da hingegen das Murmelthier weiter gegen Süden und im Gebürge zu finden ist. Sibirien, die Ukräne, das Elsas, Deutschland, und namentlich das Herzogthum Gotha sind das Vaterland des erstern. Ausser dem verschiedenen Aufenthalt zeichnen sich aber beide Thiere auch durch ihren sehr contrastirenden Charakter aus. Das Murmelthier war kirre, sehr leicht zu zähmen ꝛc lanter gute Seiten, von denen der Hamster keine einzige hat.
(Metadata: {'source': 'Handbuch der Naturgeschichte', 'autor': 'Blumenbach', 'edition': 1, 'date': 1779, 'language': 'German', 'in-text location': 'Register 4th Chapter: Säugethiere, VI. GLIRES., 1774.'})

Das Murmelthier war kirre, sehr leicht zu zähm

#### Without register

In [12]:
vectordb_jina = FAISS.from_documents(documents = docs_ed1_700_without_register, 
                                embedding = hf_jina)

In [13]:
test_embedding(file_name = "jina", vectordb = vectordb_jina, 
               model_name = model_name, register = False)

This is a trial with jinaai/jina-embeddings-v2-base-de and without register


Question: How do hamsters compare with marmots?

In der Gestalt, wie diese Insecten, die sich einer Metamorphose unterziehen, zuerst aus dem Ey kriechen, heisen sie Larven. Theils haben sie Füsse, wie die Raupen und Engerlinge theils aber keine, wie die Maden. Flügel haben sie noch nicht. Auch sind sie in diesem Zustand zur Fortpflanzung noch gänzlich unfähig: sie ernähren sich blos, und wachsen, und häuten sich mit unter einige mal.
(Metadata: {'source': 'Handbuch der Naturgeschichte', 'autor': 'J.F.Blumenbach', 'edition': 1, 'date': 1779, 'language': 'German', 'in-text location': '§. 147.'})

Ausserdem gibt es aber noch eine Menge kleiner schorfiger moosartiger Gewächse, die schon in ihrem ganzen Bau von den übrigen abweichen und deren Fortpflanzung auf eine von den erzählten verschiedene, bis jetzt noch dunkle und nicht sattsam beobachtete Weise vor sich geht,
(cryptogamia L.) Von einigen, wie von den Pilz

### b) intfloat/multilingual-e5-large

In [22]:
model_name = "intfloat/multilingual-e5-large"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_intfloat = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)  # if this returns a RuntimeError, restart the Jupyter Notebook

#### With register

In [23]:
vectordb_intfloat = FAISS.from_documents(documents = docs_ed1_700_with_register, 
                                embedding = hf_intfloat)

In [24]:
test_embedding(file_name = "intfloat", vectordb = vectordb_intfloat, 
               model_name = model_name, register = True)

This is a trial with intfloat/multilingual-e5-large and with register


Question: How do hamsters compare with marmots?

8. Taf. I. II Die Murmelthiere und die Hamster baden in Rücksicht ihres Körperbaues und ihrer Lebensart vieles mit einander gemein. Nur bewohnt der Hamster mehr nordliche Gegenden und flaches Land; da hingegen das Murmelthier weiter gegen Süden und im Gebürge zu finden ist. Sibirien, die Ukräne, das Elsas, Deutschland, und namentlich das Herzogthum Gotha sind das Vaterland des erstern. Ausser dem verschiedenen Aufenthalt zeichnen sich aber beide Thiere auch durch ihren sehr contrastirenden Charakter aus. Das Murmelthier war kirre, sehr leicht zu zähmen ꝛc lanter gute Seiten, von denen der Hamster keine einzige hat.
(Metadata: {'source': 'Handbuch der Naturgeschichte', 'autor': 'Blumenbach', 'edition': 1, 'date': 1779, 'language': 'German', 'in-text location': 'Register 4th Chapter: Säugethiere, VI. GLIRES., 1774.'})

Gewönlich haben auch die Männchen, zu uns unbekan

#### Without register

In [25]:
vectordb_intfloat = FAISS.from_documents(documents = docs_ed1_700_without_register, 
                                embedding = hf_intfloat)

In [26]:
test_embedding(file_name = "intfloat", vectordb = vectordb_intfloat, 
               model_name = model_name, register = False)

This is a trial with intfloat/multilingual-e5-large and without register


Question: How do hamsters compare with marmots?

Gewönlich haben auch die Männchen, zu uns unbekannten Zwecken, dergleichen Brüste; doch fehlen sie einigen, wie den Hamstern, der Haselmaus u. a.m. gänzlich; bey andern sind sie doch in geringerer Anzal als der Weibchen ihre der Hund z. B. hat nur sechs Zitzen am Bauche die Hündin aber ausser diesen auch noch viere an der Brust; und allemal sind sie kleiner als beym weiblichen Geschlechte.
(Metadata: {'source': 'Handbuch der Naturgeschichte', 'autor': 'J.F.Blumenbach', 'edition': 1, 'date': 1779, 'language': 'German', 'in-text location': '§. 41.'})

Sie verkriechen sich wenn diese Zeit kommt, an sichre schaurige Orte wie die Murmelthiere, Hamster, Ameisen ꝛc in ihre Nester, die Fledermäuse in Hölen, die Frösche und einige Fische in Sümpfe, die Uferschwalben ins Schilf, die Schlangen und Schnecken ins Gebüsch u. s. w. und fallen mit einbrechender Kälte in eine Art 

### c) T-Systems-onsite/cross-en-de-roberta-sentence-transformer

In [5]:
model_name = "T-Systems-onsite/cross-en-de-roberta-sentence-transformer"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_tsys = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)  # if this returns a RuntimeError, restart the Jupyter Notebook

/Users/Onema/anaconda3/envs/4llama_py11/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /Users/Onema/.cache/torch/sentence_transformers/T-Systems-onsite_cross-en-de-roberta-sentence-transformer. Creating a new one with MEAN pooling.


#### With register

In [7]:
vectordb_tsys = FAISS.from_documents(documents = docs_ed1_700_with_register, 
                                embedding = hf_tsys)

In [8]:
test_embedding(file_name = "T-sys", vectordb = vectordb_tsys, 
               model_name = model_name, register = True)

This is a trial with T-Systems-onsite/cross-en-de-roberta-sentence-transformer and with register


Question: How do hamsters compare with marmots?

8. Taf. I. II Die Murmelthiere und die Hamster baden in Rücksicht ihres Körperbaues und ihrer Lebensart vieles mit einander gemein. Nur bewohnt der Hamster mehr nordliche Gegenden und flaches Land; da hingegen das Murmelthier weiter gegen Süden und im Gebürge zu finden ist. Sibirien, die Ukräne, das Elsas, Deutschland, und namentlich das Herzogthum Gotha sind das Vaterland des erstern. Ausser dem verschiedenen Aufenthalt zeichnen sich aber beide Thiere auch durch ihren sehr contrastirenden Charakter aus. Das Murmelthier war kirre, sehr leicht zu zähmen ꝛc lanter gute Seiten, von denen der Hamster keine einzige hat.
(Metadata: {'source': 'Handbuch der Naturgeschichte', 'autor': 'Blumenbach', 'edition': 1, 'date': 1779, 'language': 'German', 'in-text location': 'Register 4th Chapter: Säugethiere, VI. GLIRES., 1774.'})

Das Murmelthier war ki

#### Without register

In [9]:
vectordb_tsys = FAISS.from_documents(documents = docs_ed1_700_without_register, 
                                embedding = hf_tsys)

In [10]:
test_embedding(file_name = "T-sys", vectordb = vectordb_tsys, 
               model_name = model_name, register = False)

This is a trial with T-Systems-onsite/cross-en-de-roberta-sentence-transformer and without register


Question: How do hamsters compare with marmots?

Zu solchen Naturspielen gehören z. B. die Graptolithen, Buchstabensteine, Kreuzsteine ꝛc. die Ingwersteine nemlich Mergelnüsse, die einer Ingwerwurzel ähneln. Der Confect von Tivoli, die Melonen vom Berg Carmel, die Frankenberger Kornähren, gewissermassen auch die Dendriten, der Florentiner Ruinen Marmor, die Incrustate u. s. w. Ferner die Klappersteine, Adlersteine Aetiten, das heißt hole Mergelnüsse oder Feuersteine, oder Eisenschüssige Steine in deren Höhlung lockere Steinchen befindlich sind, die folglich wenn man den Stein schüttelt, klappern müssen.
(Metadata: {'source': 'Handbuch der Naturgeschichte', 'autor': 'J.F.Blumenbach', 'edition': 1, 'date': 1780, 'language': 'German', 'in-text location': '§. 246.'})

Mehrere Muschelarten führen Perlen. Verschiedne Schneckgen cursiren bey einigen wilden Völkern statt Geldes. Die Mahlermusc